Referensi
1. 

Atribusi
1. 

In [192]:
# Mengimpor library & package yang digunakan
import math
import nltk
import string
import numpy as np
import pandas as pd

from pprint import pprint
from IPython.display import display
from collections import Counter, OrderedDict

In [193]:
# Mengimpor file korpus_abstrak.csv
korpus_df = pd.read_csv('korpus_abstrak.csv')

# Menampilkan shape & display dari korpus_abstrak.csv
print(korpus_df.shape)
display(korpus_df)

(469, 5)


,Link,Abstrak,Author,Pub,Ed
0,detail.jsp?id=46123&lokasi=lokal,Analisis Faktor yang Memengaruhi Loyalitas Kon...,Sumarliyanti,"Publisher: Depok: Fakultas Ilmu Komputer UI, ...",Putu Wuri Handayani
1,detail.jsp?id=46126&lokasi=lokal,Analisis Perilaku Pengguna Dalam Mencapai Kred...,S. Reyneta Carissa Anwar,Publisher: Depok: Fakultas Ilmu Komputer Univ...,Putu Wuri Handayani
2,detail.jsp?id=46149&lokasi=lokal,Desain Interaksi Aplikasi untuk Mengakomodasi ...,Mila Alief Alya,"Publisher: Depok"": Fakultas Ilmu Komputer Uni...",Harry Budi Santoso S
3,detail.jsp?id=46167&lokasi=lokal,Evaluasi dan Pengembangan Alternatif Desain In...,Ivana Putri,Publisher: Depok: Fakultas Ilmu Komputer Univ...,Harry Budi Santoso
4,detail.jsp?id=45008&lokasi=lokal,Evaluasi Desain Materi Instruksional Berdasark...,Aulia Chairunisa,"Publisher: Depok: Fakultas Ilmu Komputer UI, ...",Harri Budi Santoso
...,...,...,...,...,...
464,detail.jsp?id=46801&lokasi=lokal,Visualisasi Informasi dari Kemampuan Metakogni...,Resthu Rizfarchan Adisurya,Publisher: Depok: Fakultas Ilmu Komputer Univ...,Harry Budi Santoso
465,detail.jsp?id=4668&lokasi=lokal,Masalah transportasi merupakan suatu masalah y...,Hendrie Gunawan,"Publisher: Depok: Fasilkom UI, 2002 Call Numb...",Widijanto Nugroho
466,detail.jsp?id=4647&lokasi=lokal,Pergerakan benda di atas permukaan es dapat me...,Anita,"Publisher: Depok: Fasilkom UI, 2001 Call Numb...",Widijanto Nugroho
467,detail.jsp?id=4490&lokasi=lokal,"Dewasa ini, bentuk visual mendapat perhatian d...","Helmy A, Muhammad","Publisher: Depok: Fasilkom UI, 1996 Call Numb...",Eko Budiardjo


### **A - Preprocessing (5 Poin)**

Pada bagian ini, Anda diminta untuk melakukan preprocessing pada korpus Anda. Gunakan hasil preprocessing pada bagian-bagian setelah ini

1. [3] Lakukan lowercasing dan tokenisasi. Lowercasing dilakukan dengan fungsi lower() dan tokenisasi dilakukan dengan word_tokenize dari NLTK. Tampilkan 10 abstrak teratas!

In [194]:
# Mengekstrak kolom Abstrak dari korpus
abstrak_sr = korpus_df['Abstrak']

# Melakukan uncasing
abstrak_sr_cleaned = abstrak_sr.str.lower()

# Melakukan tokenisasi
abstrak_sr_cleaned_tokenized = abstrak_sr_cleaned.map(
    lambda abstract: nltk.word_tokenize(abstract)
)

# Menampilkan 10 baris pertama
abstrak_sr_cleaned_tokenized.head(10)

0    [analisis, faktor, yang, memengaruhi, loyalita...
1    [analisis, perilaku, pengguna, dalam, mencapai...
2    [desain, interaksi, aplikasi, untuk, mengakomo...
3    [evaluasi, dan, pengembangan, alternatif, desa...
4    [evaluasi, desain, materi, instruksional, berd...
5    [pengaruh, efek, moderasi, social, feature, da...
6    [rancangan, dan, implementasi, teknologi, intr...
7    [pentingnya, penerapan, teknologi, informasi, ...
8    [analisis, kegiatan, berbagi, pengetahuan, pad...
9    [adaptasi, expectation-confirmation, model, un...
Name: Abstrak, dtype: object

2. [2] Lakukan penghapusan semua karakter selain alfanumerik menggunakan
fungsi isalnum(). Tampilkan 10 abstrak teratas!

In [195]:
# Membersihkan karakter selain alfanumerik
for index, abstrak in abstrak_sr_cleaned_tokenized.iteritems():
    abstrak_sr_cleaned_tokenized.iloc[index] = [word for word in abstrak if word.isalnum()]

abstrak_sr_cleaned_tokenized.head(10)

0    [analisis, faktor, yang, memengaruhi, loyalita...
1    [analisis, perilaku, pengguna, dalam, mencapai...
2    [desain, interaksi, aplikasi, untuk, mengakomo...
3    [evaluasi, dan, pengembangan, alternatif, desa...
4    [evaluasi, desain, materi, instruksional, berd...
5    [pengaruh, efek, moderasi, social, feature, da...
6    [rancangan, dan, implementasi, teknologi, intr...
7    [pentingnya, penerapan, teknologi, informasi, ...
8    [analisis, kegiatan, berbagi, pengetahuan, pad...
9    [adaptasi, model, untuk, memprediksi, pengaruh...
Name: Abstrak, dtype: object

### **B - BM-25 (30 Poin)**

Pada bagian ini, Anda akan dipandu untuk melakukan implementasi retrieval menggunakan BM25 secara langkah demi langkah pada korpus yang telah melalui tahapan pre-processing

1. [2] Untuk setiap abstrak yang telah di-preprocessing, buatlah dictionary dengan key berupa token dan value berupa term frequency. Anda diperkenankan menggunakan library collections, namun Anda tidak wajib menggunakan library ini. Tampilkan dictionary untuk masing-masing dari 10 abstrak teratas

In [196]:
abstrak_sr_preprocessed = abstrak_sr_cleaned_tokenized.copy(deep=True)
N = abstrak_sr_preprocessed.size

# Menghitung total word freq dari abstrak
abstrak_dct_word_freq = Counter()
# Menghitung term freq (tf) dari abstrak
abstrak_dct_tf = dict()

print_counter = 0
for index, abstrak in abstrak_sr_preprocessed.iteritems():
    word_freq = Counter(abstrak)
    abstrak_dct_word_freq += + word_freq
    abstrak_dct_tf[index] = word_freq

    # Mencetak 10 dictionary dari 10 abstrak teratas
    if print_counter < 10:
        print(word_freq, "\n")
        print_counter += 1

Counter({'yang': 10, 'value': 7, 'faktor': 5, 'memengaruhi': 5, 'di': 4, 'dan': 4, 'dalam': 3, 'model': 3, 'perceived': 3, 'secara': 3, 'positif': 3, 'terhadap': 3, 'mana': 3, 'paling': 3, 'kuat': 3, 'satisfaction': 3, 'analisis': 2, 'loyalitas': 2, 'konsumen': 2, 'makanan': 2, 'pada': 2, 'online': 2, 'penelitian': 2, 'yaitu': 2, 'teori': 2, 'diperoleh': 2, 'adalah': 2, 'data': 2, 'emotional': 2, 'fulfilment': 2, 'membeli': 1, 'layanan': 1, 'studi': 1, 'kasus': 1, 'ini': 1, 'dilakukan': 1, 'untuk': 1, 'menganalisis': 1, 'pembelian': 1, 'indonesia': 1, 'digunakan': 1, 'diajukan': 1, 'berasal': 1, 'dari': 1, 'tiga': 1, 'kualitas': 1, 'hubungan': 1, 'perval': 1, 'mobile': 1, 'service': 1, 'quality': 1, 'total': 1, 'responden': 1, 'berhasil': 1, 'sebanyak': 1, '852': 1, 'lengkap': 1, 'tersebut': 1, 'dianalisis': 1, 'menggunakan': 1, 'pendekatan': 1, 'kuantitatif': 1, 'dengan': 1, 'metode': 1, 'structural': 1, 'equation': 1, 'didukung': 1, 'oleh': 1, 'tools': 1, 'amos': 1, 'hasil': 1, 'soci

2. [6] Buatlah daftar vocabulary (token yang unik) dari seluruh abstrak dan hitung document frequency serta inverse document frequency (idf) untuk setiap vocabulary

In [197]:
# Menghitung unique word freq (df) dari abstrak
abstrak_dct_df = Counter()

for abstrak in abstrak_sr_preprocessed:
    word_freq = Counter(set(abstrak))
    abstrak_dct_df += + word_freq

pprint(abstrak_dct_df)

Counter({'yang': 466,
         'dan': 461,
         'ini': 454,
         'untuk': 453,
         'dengan': 453,
         'pada': 390,
         'dalam': 376,
         'dari': 372,
         'menggunakan': 351,
         'penelitian': 340,
         'dapat': 336,
         'hasil': 315,
         'adalah': 307,
         'tersebut': 282,
         'dilakukan': 272,
         'di': 249,
         'bahwa': 246,
         'digunakan': 238,
         'metode': 221,
         'data': 218,
         'oleh': 212,
         'terhadap': 204,
         'satu': 196,
         'merupakan': 193,
         'sistem': 192,
         'sebagai': 191,
         'secara': 189,
         'melakukan': 183,
         'yaitu': 180,
         'itu': 177,
         'menunjukkan': 177,
         'tidak': 174,
         'salah': 173,
         'juga': 169,
         'lebih': 169,
         'telah': 160,
         'suatu': 156,
         'indonesia': 153,
         'proses': 148,
         'memiliki': 146,
         'menjadi': 145,
         'bertuju

In [198]:
# Menghitung inverse document freq (idf) dari abstrak
abstrak_dct_idf = Counter()

def count_idf(N: int, df: int) -> int:
    """Fungsi menghitung IDF"""
    return math.log(1 + ((N - df + 0.5)/(df + 0.5)))

for word, df in abstrak_dct_df.items():
    abstrak_dct_idf[word] = count_idf(N, df)

pprint(abstrak_dct_idf)

Counter({'perval': 5.74726758659594,
         'fulfilment': 5.74726758659594,
         'contact': 5.74726758659594,
         'emotional': 5.74726758659594,
         '852': 5.74726758659594,
         '341': 5.74726758659594,
         'affection': 5.74726758659594,
         'dual': 5.74726758659594,
         'normative': 5.74726758659594,
         'informational': 5.74726758659594,
         'showing': 5.74726758659594,
         'indikasi': 5.74726758659594,
         'ketertinggalan': 5.74726758659594,
         'cina': 5.74726758659594,
         'terhindar': 5.74726758659594,
         'praktisi': 5.74726758659594,
         'kemunculannya': 5.74726758659594,
         'diklaim': 5.74726758659594,
         'depannya': 5.74726758659594,
         'affective': 5.74726758659594,
         '654': 5.74726758659594,
         'appraisal': 5.74726758659594,
         'terkendali': 5.74726758659594,
         'firebase': 5.74726758659594,
         'mengunci': 5.74726758659594,
         'voting': 5.747267

3. [4] Carilah document length (dl) dari setiap abstrak dan average document length (adl) dari keseluruhan abstrak. Tampilkan nilai dl dari 10 abstrak teratas.

In [199]:
# Menghitung document length (dl) dari setiap abstrak
abstrak_dct_dl = dict()
abstrak_total_word_length = 0

for index, abstrak in abstrak_sr_preprocessed.iteritems():
    word_length = len(abstrak)
    abstrak_dct_dl[index] = word_length
    abstrak_total_word_length += word_length

pprint(abstrak_dct_dl)

{0: 148,
 1: 105,
 2: 124,
 3: 138,
 4: 126,
 5: 108,
 6: 185,
 7: 135,
 8: 151,
 9: 278,
 10: 117,
 11: 165,
 12: 156,
 13: 200,
 14: 159,
 15: 257,
 16: 239,
 17: 172,
 18: 175,
 19: 169,
 20: 161,
 21: 358,
 22: 111,
 23: 154,
 24: 151,
 25: 162,
 26: 135,
 27: 323,
 28: 231,
 29: 252,
 30: 238,
 31: 243,
 32: 170,
 33: 172,
 34: 221,
 35: 163,
 36: 240,
 37: 180,
 38: 213,
 39: 138,
 40: 125,
 41: 142,
 42: 154,
 43: 269,
 44: 182,
 45: 234,
 46: 163,
 47: 160,
 48: 110,
 49: 249,
 50: 206,
 51: 168,
 52: 119,
 53: 289,
 54: 199,
 55: 154,
 56: 260,
 57: 125,
 58: 221,
 59: 117,
 60: 139,
 61: 165,
 62: 209,
 63: 118,
 64: 125,
 65: 153,
 66: 128,
 67: 118,
 68: 158,
 69: 110,
 70: 105,
 71: 113,
 72: 203,
 73: 226,
 74: 205,
 75: 228,
 76: 204,
 77: 196,
 78: 144,
 79: 171,
 80: 231,
 81: 154,
 82: 114,
 83: 140,
 84: 125,
 85: 241,
 86: 170,
 87: 110,
 88: 179,
 89: 197,
 90: 97,
 91: 165,
 92: 196,
 93: 220,
 94: 170,
 95: 102,
 96: 111,
 97: 183,
 98: 103,
 99: 169,
 100: 95,
 

In [200]:
# Menghitung average document length (adl)
abstrak_adl = abstrak_total_word_length / N

print(f"Average document length (adl) keseluruhan adalah: {abstrak_adl}")

Average document length (adl) keseluruhan adalah: 177.03411513859274


4. [12] Buatlah fungsi score(query, doc, k, b) yang memberi output nilai skor relevansi antara query dengan masing-masing abstrak (doc).

In [201]:
# Membuat fungsi score
def score(query: str, doc: int, k: float, b: float) -> float:
    """
    Fungsi score untuk mencari relevansi query dgn doc

    Menggunakan algoritma BM25
    """
    adl = abstrak_adl
    dl = abstrak_dct_dl[doc]

    score_count = 0
    for word in nltk.word_tokenize(query):
        tf = abstrak_dct_tf[doc].get(word, 0)
        idf = abstrak_dct_idf.get(word, 0)

        score_count += (
            (tf / (tf + k * (1-b + (b*dl/adl)))) * idf
        )

    return score_count

5. [6] Gunakan fungsi score yang telah Anda buat untuk mencari 10 abstrak dengan
relevansi tertinggi untuk query, nilai k, dan b berikut ini:

    a. query: information retrieval, k=1.2, b=0.75

    b. query: sistem manajemen pengetahuan, k=1.2, b=0.5

    c. query: knowledge discovery, k=2, b=0.5

    Tampilkan 10 tokenized abstrak dengan nilai relevansi tertinggi untuk masing-masing dari (a), (b), dan (c). Anda dilarang menggunakan library yang merupakan implementasi BM25 secara langsung.

In [202]:
from typing import Union

def top_10_tokenized_abstrak(query_dct_score: Union[dict, Counter]) -> OrderedDict:
    """Helper function untuk mencetak 10 tokenized abstrak dengan skor tertinggi"""
    tokenized_dct = OrderedDict()

    for pair in query_dct_score.most_common(10):
        index = pair[0]
        tokenized_dct[index] = abstrak_sr_preprocessed[index]

    return tokenized_dct

def top_10_abstrak_with_bm25(query: str, k: float, b: float) -> float:
    """
    Mencari 10 abstrak dengan nilai relevansi tinggi

    Menggunakan algoritma BM25
    """
    query_dct_score = Counter()
    for key in abstrak_sr_preprocessed.keys():
        relevancy_score = score(query, key, k, b)
        query_dct_score[key] = relevancy_score

    print("Hasil penghitungan skor; 10 dokumen abstrak dengan skor tertinggi:")
    pprint(query_dct_score.most_common(10))

    print("\n Tokenized abstrak; 10 dokumen abstrak dengan skor tertinggi")
    pprint(top_10_tokenized_abstrak(query_dct_score))

    a. query: information retrieval, k=1.25, b=0.5

In [203]:
query = "information retrieval"
k = 1.2
b = 0.5

top_10_abstrak_with_bm25(query, k, b)

Hasil penghitungan skor; 10 dokumen abstrak dengan skor tertinggi:
[(397, 4.638354110631741),
 (188, 3.2496748549929393),
 (381, 3.2448540296418122),
 (50, 2.6422877922810177),
 (36, 2.6060983698071443),
 (131, 2.3815803526404995),
 (110, 2.1369264064414444),
 (416, 2.024226241123373),
 (143, 1.9936602770026055),
 (19, 1.97449349385729)]

 Tokenized abstrak; 10 dokumen abstrak dengan skor tertinggi
OrderedDict([(397,
              ['perolehan',
               'informasi',
               'lintas',
               'bahasa',
               'menggunakan',
               'korpus',
               'paralel',
               'dengan',
               'metode',
               'bilingual',
               'mapping',
               'perolehan',
               'informasi',
               'lintas',
               'bahasa',
               'atau',
               'cross',
               'language',
               'information',
               'retrieval',
               'clir',
               'merupakan',

    b. query: sistem manajemen pengetahuan, k=1.2, b=0.5

In [204]:
query = "sistem manajemen pengetahuan"
k = 1.2
b = 0.5

top_10_abstrak_with_bm25(query, k, b)

Hasil penghitungan skor; 10 dokumen abstrak dengan skor tertinggi:
[(461, 3.610303430991813),
 (240, 3.2818642003197174),
 (453, 3.2486817607270457),
 (264, 3.122077814893121),
 (249, 3.0909969707052136),
 (418, 2.8594785137556293),
 (15, 2.807410685726007),
 (399, 2.806054112969162),
 (8, 2.5235585560437026),
 (43, 2.4639038341995807)]

 Tokenized abstrak; 10 dokumen abstrak dengan skor tertinggi
OrderedDict([(461,
              ['usability',
               'evaluation',
               'sistem',
               'informasi',
               'manajemen',
               'rumah',
               'sakit',
               'studi',
               'kasus',
               'aplikasi',
               'instalasi',
               'gawat',
               'darurat',
               'rumah',
               'sakit',
               'umum',
               'daerah',
               'pasar',
               'minggu',
               'penggunaan',
               'teknologi',
               'informasi',
           

    c. query: knowledge discovery, k=2, b=0.5

In [205]:
query = "knowledge discovery"
k = 2
b = 0.5

top_10_abstrak_with_bm25(query, k, b)

Hasil penghitungan skor; 10 dokumen abstrak dengan skor tertinggi:
[(138, 2.9539109682102906),
 (211, 2.9041448463244097),
 (417, 2.8378267847718694),
 (43, 2.613106781591238),
 (373, 2.506726035328042),
 (240, 2.3044211268834562),
 (421, 1.7584333553596834),
 (110, 1.3885203021103327),
 (376, 1.164513097467822),
 (158, 1.101996061286173)]

 Tokenized abstrak; 10 dokumen abstrak dengan skor tertinggi
OrderedDict([(138,
              ['yang',
               'mempengaruhi',
               'kesuksesan',
               'implementasi',
               'knowledge',
               'management',
               'di',
               'pt',
               'jasa',
               'marga',
               'persero',
               'tbk',
               'perkembangan',
               'ilmu',
               'pengetahuan',
               'dan',
               'teknologi',
               'menyebabkan',
               'meningkatnya',
               'daya',
               'saing',
               'antar',
   

### **C- Neural Embedding Word2Vec (30 Poin)**

1. [2] Menggunakan data preprocessing, lakukan stemming dengan menggunakan library Sastrawi untuk masing-masing token pada setiap baris data.

2. [5] Menggunakan data hasil stemming, buatlah sebuah model Word2Vec dengan modul yang disediakan oleh gensim (silahkan baca dokumentasi lebih lanjut) dengan ketentuan sebagai berikut:

    a. ukuran dimensi Word2Vec adalah 2

    b. dibebaskan untuk menggunakan model skip-gram ataupun CBOW

    c. - jumlah iterasi (epoch) sebesar 10.

    d. untuk parameter lainnya dibebaskan kepada mahasiswa

3. [2] Gunakan model yang Anda buat untuk mencari representasi word2vec dari term berikut ini:

    a. algoritma

    b. *interface*

4. [3] Plot nilai vektor dari kedua term yang teman-teman dapatkan (soal nomor 3) dalam grafik 2 dimensi euclidean space (tidak perlu menggunakan dimensionality reduction karena vektor sudah berada pada 2 dimensi).

    **DISCLAIMER**: Hasil plot yang didapat mungkin saja tidak sesuai dengan ekspektasi teman - teman mengingat kecilnya nilai epoch yang digunakan dan korpus yang digunakan untuk kebutuhan tugas ini.

5. [12] Buatlah sebuah fungsi yang dapat mengembalikan dokumen relevan menggunakan Word2Vec berdasarkan abstrak jika diberikan sebuah query. Cara kerjanya dapat tapi tidak terbatas seperti berikut:

    a. Untuk menghitung vektor suatu dokumen atau kumpulan kata dapat dilakukan dengan menggabungkan vektor kata (word embedding). Penggabungan juga dapat bervariasi.

    b. Mencari dokumen yang relevan dapat dilakukan dengan menghitung similarity (contoh: cosine similarity) antara representasi vektor query dan vektor dokumen. Seharusnya dokumen dengan similarity tinggi cenderung memiliki relevansi yang tinggi juga.

6. [6] Gunakan fungsi yang telah Anda buat untuk mencari 10 abstrak dengan relevansi tertinggi untuk query:

   a. query: information retrieval

   b. query: sistem manajemen pengetahuan

   c. query: knowledge discovery

7. [10] **BONUS**: Anda dapat menggunakan pretrained model word2vec yang sudah dilatih menggunakan korpus wikipedia bahasa Indonesia. Setelah berhasil me-load model tersebut, Anda dapat mencoba menggunakan model tersebut untuk menjawab soal nomor 5 dan 6. Coba ceritakan perbedaan apa yang teman-teman dapatkan tanpa dan dengan pretrained model?

### **D - Dimensionality Reduction (10 Poin)**

1. [2] Buatlah representasi matriks TF-IDF dengan mengalikan nilai tf pada setiap terms di setiap dokumen dan idf dari dokumen tersebut. Gunakan nilai tf dan idf yang Anda dapatkan pada nomor B2. Kemudian tampilkan hasilnya.

2. [3] Berdasarkan hasil yang sudah didapatkan pada nomor 1, lakukan pemprosesan menggunakan PCA (2 komponen), kemudian tampilkan seluruh dokumen dalam bentuk visualisasi PCA di plot euclidean space.

    **Tips**: Bayangkan setiap kata pada korpus berada di posisi kolom, sedangkan nomor dokumen berada di posisi baris, kemudian semua kata yang ada pada korpus akan direduksi menjadi 2 principal component.

3. [3] Berdasarkan hasil yang sudah didapatkan pada nomor 1, lakukan pemprosesan menggunakan SVD (2 komponen). Anda dapat mencoba menggunakan **TruncatedSVD** pada library scikit-learn untuk menyelesaikannya.

4. [2] Bagaimana hasil yang Anda dapatkan setelah menggunakan PCA dan SVD ? Apakah ada hal unik yang bisa Anda temukan ? Jelaskan analisis singkat Anda minimal 3 kalimat.

### **E - IR Model Evaluation (25 Poin)**

Pada bagian ini, Anda diminta untuk mengevaluasi hasil retrieval dari sistem yang telah Anda buat pada bagian B dan C dan memberikan analisis Anda berdasarkan hasil evaluasi yang Anda dapatkan. Gunakan hanya query “information retrieval” (query nomor a pada B5 dan C6) di bagian ini.

1. [5] Lakukan human judgment (proses manual dalam menilai apakah dokumen yang diperoleh merupakan dokumen yang relevan dengan query yang diberikan) pada 10 abstrak untuk masing-masing hasil retrieval menggunakan BM25 (bagian B) dan Word2Vec (bagian C). Berikan nilai 1 jika Anda anggap abstrak tersebut relevan dengan query awal dan 0 jika tidak. Untuk tugas ini, tidak masalah jika Anda tidak mengetahui secara pasti apakah query benar-benar relevan dengan dokumen yang diperoleh. Yang penting adalah Anda konsisten dalam menilai kedua hasil retrieval. Tampilkan hasil human judgement yang telah Anda buat!

2. [10] Berdasarkan hasil dari nomor 1, hitung P@1 (precision-at-1), P@3, P@5, Mean Average Precision (MAP) dengan micro average, dan MAP dengan macro average dari hasil retrieval Anda untuk BM25 dan Word2Vec. Anda boleh melakukan perhitungan ini secara manual ataupun otomatis.

3. [7] Andaikan pada ground truth terdapat 5 item yang relevan dan berdasarkan hasil dari nomor 1, hitung R@1 (recall-at-1), R@3, R@5 dari hasil retrieval Anda untuk BM25 dan Word2Vec. Anda boleh melakukan ini secara manual maupun otomatis.

4. [3] Berdasarkan hasil dari nomor-nomor sebelumnya di bagian ini, lakukan analisis perbandingan mana yang lebih baik antara model BM25 atau Word2Vec. Sertakan penjelasan singkat dalam 3-5 kalimat terkait hasil yang didapatkan tersebut.